In [169]:
%load_ext autoreload
%autoreload 2
import sys
import os
import argparse
import numpy as np
from tqdm.auto import tqdm
import spectrumMatching
import pyopenms as pyms

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of spectrumMatching failed: Traceback (most recent call last):
  File "/Users/reder/miniconda3/envs/metabolomics/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/Users/reder/miniconda3/envs/metabolomics/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/Users/reder/miniconda3/envs/metabolomics/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 846, in exec_module
  File "<frozen importlib._bootstrap_external>", line 983, in get_code
  File "<frozen importlib._bootstrap_external>", line 913, in source_to_code
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/Users/reder/MSChemUtils/spectrumMatching.py", line 62
    mse = pyms.M

In [222]:
import importlib
spectrumMatching = importlib.reload(spectrumMatching)

class argsObject(object):
    pass
args = argsObject()
args.tsv = "/Users/reder/Library/CloudStorage/OneDrive-Chalmers/MS-MS_Comparison/210128_MS-MS_Comparison_Algorithm/Software_deliverables/2023-05-26_CDF_test/2023-05-26_CDF_test.tsv"
args.random_seed = 1
with open(args.tsv, 'r') as f:
    lines = f.readlines()

for i_line, line in enumerate(tqdm(lines)):
    if i_line == 0:
        header = line.split('\t')
        continue
    line_args = line.split('\t')
    cmd_args_dict = dict()
    for (h, a) in zip(header, line_args):
        if a not in ["", '\n']:
            cmd_args_dict[h] = a
    cmd_args_dict['parentMZ'] = cmd_args_dict['Exact m/z']
    cmd_args_dict['parentFormula'] = cmd_args_dict['Formula (w/ adduct)']
    if "inDir" in cmd_args_dict.keys():
        if "inFiles" in cmd_args_dict.keys():
            sys.exit(f"Both inDir and inFiles were provided in row {i_line + 1}. Please provide only one")
        idt = cmd_args_dict['inDir']
        cmd_args_dict['inFiles'] = ",".join([os.path.join(idt, x) for x in os.listdir(idt) if x.lower().endswith('.mzml')])
    hit_rows = spectrumMatching.scrape_spectra_hits(**cmd_args_dict)
    break

 50%|█████     | 1/2 [00:29<00:29, 29.23s/it]


In [223]:
# hit_rows
np.random.seed(args.random_seed)
choices = np.random.choice(range(0, len(hit_rows)), len(hit_rows), replace=False)
i = 0
pairs = []
while True:
    pairs.append((choices[i], choices[i + 1]))
    i += 2
    if i >= len(choices):
        break

In [224]:
for iPair, (iSpec, jSpec) in enumerate(pairs):
    iMeta = hit_rows[iSpec]
    jMeta = hit_rows[jSpec]
    drop_keys = ["Compound", "inDir", "inFiles", "Mode", "Collision Energy", 
                "Target RT", "Begin", "End", "Adduct", "Formula (w/ adduct)",
                "Exact m/z", "Targeted m/z", "isolationMZTol"]
    spec_matching_args_dict = {k : v for (k,v) in cmd_args_dict.items() if k not in drop_keys}
    spec_matching_args_dict['outDir'] = os.path.join(cmd_args_dict['outDir'], "spectrum_matching")
    spec_matching_args_dict['outPrefix'] = f"{cmd_args_dict['Compound']}_{cmd_args_dict['Mode']}_{cmd_args_dict['Collision Energy']}_{iPair}"
    spec_matching_args_dict['mzml1'] = iMeta[0]
    spec_matching_args_dict['mzml2'] = jMeta[0]
    spec_matching_args_dict['index1'] = iMeta[2]
    spec_matching_args_dict['index2'] = jMeta[2]
    spec_matching_args_dict['startingIndex'] = 0
    spec_matching_args = " ".join([f"--{k} {v}" for (k, v) in spec_matching_args_dict.items()])
    spec_matching_args = spectrumMatching.get_args(spec_matching_args)
    break

In [225]:
spectrumMatching.run_matching(spec_matching_args)

mkdir -p /Users/reder/OneDrive/MS-MS_Comparison/210128_MS-MS_Comparison_Algorithm/Software_deliverables/2023-05-26_CDF_test/Caffeine_Pos_20V/spectrum_matching
---- Prepping Spectrum 1 ----
Filtering peaks....
---- Prepping Spectrum 2 ----
Filtering peaks....


In [218]:
spec_matching_args

Namespace(mzml1='/Users/reder/OneDrive/MS-MS_Comparison/210128_MS-MS_Comparison_Algorithm/6530_Q-TOF/211219_MSMS_Mix_Repeats/MSMS_Pos_0-60V/20V_08.mzML', index1=1032, mzml2='/Users/reder/OneDrive/MS-MS_Comparison/210128_MS-MS_Comparison_Algorithm/6530_Q-TOF/211219_MSMS_Mix_Repeats/MSMS_Pos_0-60V/20V_45.mzML', index2=1040, quasiX=114.5, quasiY=-0.238, absCutoff=0, relCutoff=0, DUMin=-0.5, PEL=None, PDPL=None, startingIndex=0, R=10000.0, gainControl=False, quasiCutoff=2.0, minSpectrumQuasiCounts=20, minTotalPeaks=2, outDir='/Users/reder/OneDrive-Chalmers/MS-MS_Comparison/210128_MS-MS_Comparison_Algorithm/Software_deliverables/2023-05-26_CDF_test/Caffeine_Pos_20V/spectrum_matching', outPrefix='Caffeine_Positive_20V_0', parentMZ=195.08765, parentFormula='C8H11N4O2', resClearance=0.02, matchAcc=0.01, subFormulaTol=0.01)

In [215]:
spec_matching_args_dict

{'outDir': '/Users/reder/OneDrive-Chalmers/MS-MS_Comparison/210128_MS-MS_Comparison_Algorithm/Software_deliverables/2023-05-26_CDF_test/Caffeine_Pos_20V/spectrum_matching',
 'R': '10000',
 'startingIndex': '1',
 'quasiX': '114.5',
 'quasiY': '-0.238',
 'quasiCutoff': '2',
 'parentMZ': '195.08765',
 'parentFormula': 'C8H11N4O2',
 'outPrefix': 'Caffeine_Positive_20V_0',
 'mzml1': '/Users/reder/OneDrive/MS-MS_Comparison/210128_MS-MS_Comparison_Algorithm/6530_Q-TOF/211219_MSMS_Mix_Repeats/MSMS_Pos_0-60V/20V_08.mzML',
 'mzml2': '/Users/reder/OneDrive/MS-MS_Comparison/210128_MS-MS_Comparison_Algorithm/6530_Q-TOF/211219_MSMS_Mix_Repeats/MSMS_Pos_0-60V/20V_45.mzML',
 'index1': 1032,
 'index2': 1040}

In [206]:
spec_matching_args_dict

{'outDir': '/Users/reder/OneDrive-Chalmers/MS-MS_Comparison/210128_MS-MS_Comparison_Algorithm/Software_deliverables/2023-05-26_CDF_test/Caffeine_Pos_20V/spectrum_matching',
 'R': '10000',
 'startingIndex': '1',
 'quasiX': '114.5',
 'quasiY': '-0.238',
 'quasiCutoff': '2',
 'parentMZ': '195.08765',
 'parentFormula': 'C8H11N4O2',
 'mzml1': '/Users/reder/OneDrive/MS-MS_Comparison/210128_MS-MS_Comparison_Algorithm/6530_Q-TOF/211219_MSMS_Mix_Repeats/MSMS_Pos_0-60V/20V_08.mzML',
 'mzml2': '/Users/reder/OneDrive/MS-MS_Comparison/210128_MS-MS_Comparison_Algorithm/6530_Q-TOF/211219_MSMS_Mix_Repeats/MSMS_Pos_0-60V/20V_45.mzML',
 'index1': 1032,
 'index2': 1040}

'/Users/reder/OneDrive-Chalmers/MS-MS_Comparison/210128_MS-MS_Comparison_Algorithm/Software_deliverables/2023-05-26_CDF_test/Caffeine_Pos_20V/spectrum_matching'

In [207]:
cmd_args_dict

{'Compound': 'Caffeine',
 'inFiles': '"/Users/reder/OneDrive/MS-MS_Comparison/210128_MS-MS_Comparison_Algorithm/6530_Q-TOF/211219_MSMS_Mix_Repeats/MSMS_Pos_0-60V/20V_01.mzML,/Users/reder/OneDrive/MS-MS_Comparison/210128_MS-MS_Comparison_Algorithm/6530_Q-TOF/211219_MSMS_Mix_Repeats/MSMS_Pos_0-60V/20V_02.mzML,/Users/reder/OneDrive/MS-MS_Comparison/210128_MS-MS_Comparison_Algorithm/6530_Q-TOF/211219_MSMS_Mix_Repeats/MSMS_Pos_0-60V/20V_03.mzML,/Users/reder/OneDrive/MS-MS_Comparison/210128_MS-MS_Comparison_Algorithm/6530_Q-TOF/211219_MSMS_Mix_Repeats/MSMS_Pos_0-60V/20V_04.mzML,/Users/reder/OneDrive/MS-MS_Comparison/210128_MS-MS_Comparison_Algorithm/6530_Q-TOF/211219_MSMS_Mix_Repeats/MSMS_Pos_0-60V/20V_05.mzML,/Users/reder/OneDrive/MS-MS_Comparison/210128_MS-MS_Comparison_Algorithm/6530_Q-TOF/211219_MSMS_Mix_Repeats/MSMS_Pos_0-60V/20V_06.mzML,/Users/reder/OneDrive/MS-MS_Comparison/210128_MS-MS_Comparison_Algorithm/6530_Q-TOF/211219_MSMS_Mix_Repeats/MSMS_Pos_0-60V/20V_07.mzML,/Users/reder/One

'Caffeine_Positive_20V_0'